In [1]:
# pip install StockNews

In [2]:
# pip install cchardet

In [3]:
import pandas as pd
import numpy as np
from stocknews import StockNews
from datetime import date
import datetime
import yfinance as yf
from tqdm import tqdm
import statistics
import cchardet

In [4]:
data = pd.read_csv('ftp://ftp.nasdaqtrader.com/symboldirectory/nasdaqtraded.txt', sep = '|')
data = data[(data.Symbol.notnull()) & (data.ETF == 'N')]
data = data[data['Round Lot Size'] == 100]
print(data.shape)

(7424, 12)


In [5]:
symbols = [i for i in data.Symbol if '$' not in i]
symbols = [i for i in symbols if '.' not in i]

In [6]:
# symbols = symbols[:100]

In [7]:
data = data[data.Symbol.isin(symbols)]

In [8]:
today = datetime.datetime.strptime('2021-01-04', '%Y-%m-%d').date() # date.today()
tomorrow = today + datetime.timedelta(days = 1)
yesterday = today + datetime.timedelta(days = -1)
# today = today.strftime('%Y-%m-%d')
print(yesterday, today, tomorrow)

2021-01-03 2021-01-04 2021-01-05


In [9]:
data.head()

,Nasdaq Traded,Symbol,Security Name,Listing Exchange,Market Category,ETF,Round Lot Size,Test Issue,Financial Status,CQS Symbol,NASDAQ Symbol,NextShares
0,Y,A,"Agilent Technologies, Inc. Common Stock",N,,N,100.0,N,NaN,A,A,N
1,Y,AA,Alcoa Corporation Common Stock,N,,N,100.0,N,NaN,AA,AA,N
5,Y,AACG,ATA Creativity Global - American Depositary Sh...,Q,G,N,100.0,N,N,NaN,AACG,N
6,Y,AACQ,Artius Acquisition Inc. - Class A Common Stock,Q,S,N,100.0,N,N,NaN,AACQ,N
7,Y,AACQU,Artius Acquisition Inc. - Unit consisting of o...,Q,S,N,100.0,N,N,NaN,AACQU,N


### Filter Universe, S&P for now

In [10]:
# snp = pd.read_excel('s&p.xlsx')

In [11]:
# data = data[data.Symbol.isin(snp.Tickers)].reset_index(drop = True)

In [12]:
# data

### Get Stock Prices from Yahoo

In [13]:
def get_Close(stock, start, days):
    
    end = start + datetime.timedelta(days = days)
    
    df = yf.download(stock, 
                          start=start, 
                          end=end, 
                          progress=True)
    return(df)

In [14]:
get_Close('TSLA', today, 5)

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-01-04,719.460022,744.489990,717.190002,729.770020,729.770020,48638200
2021-01-05,723.659973,740.840027,719.200012,735.109985,735.109985,32245200
2021-01-06,758.489990,774.000000,749.099976,755.979980,755.979980,44700000
2021-01-07,777.630005,816.989990,775.200012,816.039978,816.039978,51498900
2021-01-08,856.000000,884.489990,838.390015,880.020020,880.020020,74661000


In [15]:
# yahoo = get_Close(symbols, today, 5).T.reset_index()

In [16]:
# yahoo.to_pickle('yahoo_02-05.pkl')

In [17]:
yahoo = pd.read_pickle('yahoo_01-02.pkl')

In [18]:
yahoo_Close = yahoo[yahoo.level_0 == 'Close']
yahoo_Close = yahoo_Close.iloc[:,1:]
close_cols = ['Close_' + col.strftime('%Y-%m-%d') for col in yahoo_Close.columns if col != 'level_1']
yahoo_Close.columns = ['Symbol'] + close_cols

In [19]:
yahoo_Volume = yahoo[yahoo.level_0 == 'Volume']
yahoo_Volume = yahoo_Volume.iloc[:,1:]
close_cols = ['Volume_' + col.strftime('%Y-%m-%d') for col in yahoo_Volume.columns if col != 'level_1']
yahoo_Volume.columns = ['Symbol'] + close_cols

In [20]:
data = data.merge(yahoo_Close, on = 'Symbol', how = 'left')
data = data.merge(yahoo_Volume, on = 'Symbol', how = 'left')

In [21]:
data = data.dropna(axis = 1, how = 'all')
data = data.dropna(axis = 0, how = 'all')

In [22]:
close_cols = [col for col in data.columns if 'Close' in col]
vol_cols = [col for col in data.columns if 'Volume' in col]

data['average_close'] = data[close_cols[:1]].mean(axis = 1)
data['average_vol'] = data[vol_cols[:1]].mean(axis = 1)

# data = data[data.average_close <= 15]
# data = data[data.average_close >= 2]
print(close_cols)

['Close_2021-01-04', 'Close_2021-01-05', 'Close_2021-01-06', 'Close_2021-01-07', 'Close_2021-01-08', 'Close_2021-01-11', 'Close_2021-01-12', 'Close_2021-01-13', 'Close_2021-01-14', 'Close_2021-01-15', 'Close_2021-01-19', 'Close_2021-01-20', 'Close_2021-01-21', 'Close_2021-01-22', 'Close_2021-01-25', 'Close_2021-01-26', 'Close_2021-01-27', 'Close_2021-01-28', 'Close_2021-01-29', 'Close_2021-02-01', 'Close_2021-02-02', 'Close_2021-02-03', 'Close_2021-02-04', 'Close_2021-02-05']


In [23]:
# data = data.sort_values('average_vol', ascending = False).head(50).reset_index(drop = True)

In [24]:
data.shape

(6602, 62)

In [25]:
data.head()

,Nasdaq Traded,Symbol,Security Name,Listing Exchange,Market Category,ETF,Round Lot Size,Test Issue,Financial Status,CQS Symbol,...,Volume_2021-01-27,Volume_2021-01-28,Volume_2021-01-29,Volume_2021-02-01,Volume_2021-02-02,Volume_2021-02-03,Volume_2021-02-04,Volume_2021-02-05,average_close,average_vol
0,Y,A,"Agilent Technologies, Inc. Common Stock",N,,N,100.0,N,NaN,A,...,1923500.0,1677900.0,1822000.0,1292700.0,1489200.0,1448400.0,1251200.0,1919700.0,118.639999,2030700.0
1,Y,AA,Alcoa Corporation Common Stock,N,,N,100.0,N,NaN,AA,...,9215300.0,7028600.0,7772500.0,9127400.0,7218500.0,10453700.0,5516700.0,5235300.0,23.250000,5400000.0
2,Y,AACG,ATA Creativity Global - American Depositary Sh...,Q,G,N,100.0,N,N,NaN,...,61100.0,44800.0,104000.0,27700.0,25300.0,21200.0,314069900.0,18221500.0,1.180000,18000.0
3,Y,AACQ,Artius Acquisition Inc. - Class A Common Stock,Q,S,N,100.0,N,N,NaN,...,2641700.0,899300.0,758800.0,1323400.0,1047900.0,1072100.0,1178100.0,1055500.0,10.660000,1879700.0
4,Y,AACQU,Artius Acquisition Inc. - Unit consisting of o...,Q,S,N,100.0,N,N,NaN,...,353800.0,140500.0,19700.0,49800.0,39800.0,11900.0,201000.0,26000.0,11.300000,255600.0


In [26]:
# data.drop(['Close_Price_x','Volume_x','Close_Price_y','Volume_y'], axis = 1, inplace = True)

In [27]:
final_symbols = data.Symbol.tolist()

In [28]:
# final_symbols = ['GE', 'HLIT', 'ACOR', 'PBI', 'EXK','PRGX', 'NAVI', 'VRNA', 'GTT', 'TLSA']

In [29]:
final_symbols

['A',
 'AA',
 'AACG',
 'AACQ',
 'AACQU',
 'AACQW',
 'AAIC',
 'AAL',
 'AAMC',
 'AAME',
 'AAN',
 'AAOI',
 'AAON',
 'AAP',
 'AAPL',
 'AAT',
 'AAU',
 'AAWW',
 'AB',
 'ABB',
 'ABBV',
 'ABC',
 'ABCB',
 'ABCL',
 'ABCM',
 'ABEO',
 'ABEV',
 'ABG',
 'ABIO',
 'ABM',
 'ABMD',
 'ABNB',
 'ABR',
 'ABST',
 'ABT',
 'ABTX',
 'ABUS',
 'AC',
 'ACA',
 'ACAC',
 'ACACU',
 'ACACW',
 'ACAD',
 'ACB',
 'ACBI',
 'ACC',
 'ACCD',
 'ACCO',
 'ACEL',
 'ACER',
 'ACET',
 'ACEV',
 'ACEVU',
 'ACEVW',
 'ACGL',
 'ACGLO',
 'ACGLP',
 'ACH',
 'ACHC',
 'ACHV',
 'ACI',
 'ACIA',
 'ACIC',
 'ACIU',
 'ACIW',
 'ACKIT',
 'ACKIU',
 'ACKIW',
 'ACLS',
 'ACM',
 'ACMR',
 'ACN',
 'ACNB',
 'ACND',
 'ACOR',
 'ACP',
 'ACRE',
 'ACRS',
 'ACRX',
 'ACST',
 'ACTC',
 'ACTCU',
 'ACTCW',
 'ACTG',
 'ACU',
 'ACV',
 'ACY',
 'ADAP',
 'ADBE',
 'ADC',
 'ADCT',
 'ADERU',
 'ADES',
 'ADI',
 'ADIL',
 'ADILW',
 'ADM',
 'ADMA',
 'ADMP',
 'ADMS',
 'ADN',
 'ADNT',
 'ADNWW',
 'ADOC',
 'ADOCR',
 'ADOCW',
 'ADP',
 'ADPT',
 'ADS',
 'ADSK',
 'ADT',
 'ADTN',
 'ADTX',
 'A

### Sentiment Analysis

In [30]:
# final_symbols = ['AAPL','TSLA','MSFT','PLDR','ATOS','OLTK','BB']
# final_symbols = ['VLT', 'VTA', 'VVR', 'SIRI', 'SUP']

In [31]:
import pandas as pd
from bs4 import BeautifulSoup, SoupStrainer
import matplotlib.pyplot as plt
from urllib.request import urlopen, Request
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import lxml
import requests

In [32]:
today_form = datetime.datetime.strptime(today.strftime('%Y-%m-%d'), '%Y-%m-%d')
today_form

datetime.datetime(2021, 1, 4, 0, 0)

In [33]:
def convert_datetime(date, time):
    x = datetime.datetime.strptime(date + ' ' + time, '%b-%d-%y %I:%M%p')
    return(x)

In [34]:
def convert_date(date):
    x = datetime.datetime.strptime(date, '%b-%d-%y')
    return(x)

### Creating Function to get Sentiment

In [35]:
def get_Sentiment(news_date, stocks, articles):
     
    n = articles #the # of article headlines displayed per ticker
    tickers = stocks

    # Get Data
    finwiz_url = 'https://finviz.com/quote.ashx?t='
    news_tables = {}
    final_tickers = tickers.copy()

    for ticker in tqdm(tickers):
        try:
            url = finwiz_url + ticker
            req = Request(url=url,headers={'user-agent': 'my-app/0.0.1'}) 
            resp = urlopen(req)    
            html = BeautifulSoup(resp, features="lxml")
            news_table = html.find(id='news-table')
            news_tables[ticker] = news_table
        except:
            continue

    for ticker in tickers:
        try:
            df = news_tables[ticker]
            df_tr = df.findAll('tr')

            for i, table_row in enumerate(df_tr):
                a_text = table_row.a.text
                td_text = table_row.td.text
                td_text = td_text.strip()
        #             print(a_text,'(',td_text,')')
                if i == n-1:
                    break
        except:
            continue

    # Iterate through the news
    parsed_news = []
    news_tables = {i:v for i,v in news_tables.items() if v is not None}

    for file_name, news_table in news_tables.items():
        for x in news_table.findAll('tr'):
            text = x.a.get_text() 
            date_scrape = x.td.text.split()

            if len(date_scrape) == 1:
                time = date_scrape[0]

            else:
                date = date_scrape[0]
                time = date_scrape[1]

            ticker = file_name.split('_')[0]

            parsed_news.append([ticker, date, time, text])
    
    min_time = '01:00PM'
    max_time = '06:00PM'
    parsed_news = [i for i in parsed_news if convert_datetime(i[1],i[2]) >= convert_datetime(i[1],min_time) and convert_datetime(i[1],i[2]) <= convert_datetime(i[1],max_time) and convert_date(i[1]) == news_date]
    
    parsed_news = [i for i in parsed_news if 'ALERT' not in i[3] and 'NOTICE' not in i[3] and 'NEWS' not in i[3]]
    
#     if parsed_news != []:
#         news_df = pd.DataFrame(np.vstack(parsed_news), columns = ['Ticker', 'Date', 'Time', 'Headline'])
#         news_df_count = news_df.groupby('Ticker')['Headline'].count().reset_index()
#         news_df_count = news_df_count.sort_values('Headline', ascending = False)
#         #news_df_top = news_df_count[news_df_count.Headline >= 1]
#         final_tickers = [i for i in news_df_count.Ticker]

    # Sentiment Analysis
    analyzer = SentimentIntensityAnalyzer()

    columns = ['Ticker', 'Date', 'Time', 'Headline']
    news = pd.DataFrame(parsed_news, columns=columns)
    scores = news['Headline'].apply(analyzer.polarity_scores).tolist()

    df_scores = pd.DataFrame(scores)
    news = news.join(df_scores, rsuffix='_right')


    # View Data 
    news['Date'] = pd.to_datetime(news.Date).dt.date
    
    #Delete news targeted at multiple stocks
    news['articles_count'] = news.groupby('Headline')['Ticker'].transform('count')
    news = news[news.articles_count == 1]
    news.drop('articles_count', axis = 1, inplace = True)
    
    unique_ticker = news['Ticker'].unique().tolist()
    news_dict = {name: news.loc[news['Ticker'] == name] for name in unique_ticker}

    values = []
    final_tickers = news.Ticker.drop_duplicates().tolist()

    for ticker in final_tickers: 
        if news_dict != {}:
            dataframe = news_dict[ticker]
            dataframe = dataframe.set_index('Ticker')
            dataframe = dataframe.drop(columns = ['Headline'])
            #mean = round(dataframe['compound'].mean(), 2)
            mean = round(dataframe['pos'].mean(), 2)
            values.append(mean)

    df = pd.DataFrame(list(zip(final_tickers, values)), columns =['Ticker', 'Mean Sentiment']) 
    df = df.set_index('Ticker')
    df = df.sort_values('Mean Sentiment', ascending=False)
    return(df, news)
#     return(parsed_news)

In [40]:
# news

In [41]:
# parsed_news = get_Sentiment(today_form, final_symbols, 40)

In [42]:
# min_time = '12:00PM'
# max_time = '06:00PM'
# [i for i in parsed_news if convert_datetime(i[1],i[2]) >= convert_datetime(i[1],min_time) and convert_datetime(i[1],i[2]) <= convert_datetime(i[1],max_time) and convert_date(i[1]) == today_form]

In [43]:
def get_return(date, tomorrow):
    
    today_form = datetime.datetime.strptime(date, '%Y-%m-%d')
    sentiment, news = get_Sentiment(today_form, final_symbols, 100)
    sentiment = sentiment[sentiment['Mean Sentiment'] > 0.3]
    today = date
    #tomorrow = today + datetime.timedelta(days = 1)
    # today = today.strftime('%Y-%m-%d')
    #tomorrow = tomorrow.strftime('%Y-%m-%d')
        
    if sentiment.shape[0] != 0: 
        

        close_today = [col for col in data.columns if 'Close_' + str(today) in col]
        close_tomorrow = [col for col in data.columns if 'Close_' + str(tomorrow) in col]

        ### Check Returns and BackTest

        test = data.copy()
        test = test[['Symbol','Security Name','Round Lot Size'] + close_today + close_tomorrow]

        test['Sentiment'] = test['Symbol'].map(sentiment.to_dict()['Mean Sentiment'])
        test = test[test.Sentiment.notnull()].reset_index(drop = True)

        test['returns'] = (test[close_tomorrow[0]] - test[close_today[0]]) / test[close_today[0]]

        test = test.sort_values('Sentiment', ascending = False).reset_index(drop = True).head(5)
        test[['Sentiment','returns']] = round(test[['Sentiment','returns']], 2)

        stock_invested = test.Symbol.tolist()
        sentiment_invested = test.Sentiment.tolist()
        return_invested = test.returns.tolist()

        if return_invested != []:
            avg_return = statistics.mean(return_invested)

            print(f"Investing in stocks {stock_invested} on {today} with sentiments {sentiment_invested} and returns {return_invested} on {tomorrow}")
            print(f"Average return for the day {tomorrow} is {avg_return}")
        else:
            print(f"No Trades placed on {today}")
            avg_return = 0
    
    else:
        print(f"No Trades placed on {today} as no sentiments found")
        avg_return = 0
    
    return(avg_return, news)

### Check Returns and BackTest

In [47]:
# pip install pandas_market_calendars

In [48]:
import pandas_market_calendars as mcal
nyse = mcal.get_calendar('NYSE')
early = nyse.schedule(start_date='2021-01-01', end_date='2021-02-05').index.tolist()
dates = [i.strftime('%Y-%m-%d') for i in early]
dates

['2021-01-04',
 '2021-01-05',
 '2021-01-06',
 '2021-01-07',
 '2021-01-08',
 '2021-01-11',
 '2021-01-12',
 '2021-01-13',
 '2021-01-14',
 '2021-01-15',
 '2021-01-19',
 '2021-01-20',
 '2021-01-21',
 '2021-01-22',
 '2021-01-25',
 '2021-01-26',
 '2021-01-27',
 '2021-01-28',
 '2021-01-29',
 '2021-02-01',
 '2021-02-02',
 '2021-02-03',
 '2021-02-04',
 '2021-02-05']

In [49]:
Investment = 1000

In [ ]:
all_news = {}

for i in range(len(dates) - 2):
    print(f"Looking for articles on {dates[i]} for Investment")
    curr_return, all_news[dates[i]] = get_return(dates[i], dates[i+2])
    Investment = Investment * (1 + curr_return)
    print(f"Balance on {dates[i+1]} is {Investment}\n")

  0%|          | 0/6602 [00:00<?, ?it/s]

Looking for articles on 2021-01-04 for Investment


  0%|          | 0/6602 [00:00<?, ?it/s]

Investing in stocks ['CREX', 'HLT', 'AXP', 'WDAY', 'WRK'] on 2021-01-04 with sentiments [0.63, 0.61, 0.5, 0.48, 0.47] and returns [-0.07, 0.03, 0.04, -0.04, 0.09] on 2021-01-06
Average return for the day 2021-01-06 is 0.009999999999999998
Balance on 2021-01-05 is 1010.0

Looking for articles on 2021-01-05 for Investment


  0%|          | 0/6602 [00:00<?, ?it/s]

...

  0%|          | 0/6602 [00:00<?, ?it/s]

Investing in stocks ['CP', 'ORAN', 'UNFI', 'NUE', 'SLF'] on 2021-01-28 with sentiments [0.58, 0.54, 0.52, 0.48, 0.43] and returns [-0.03, -0.01, 0.01, -0.03, 0.0] on 2021-02-01
Average return for the day 2021-02-01 is -0.012
Balance on 2021-01-29 is 1086.2173968477944

Looking for articles on 2021-01-29 for Investment


  0%|          | 0/6602 [00:00<?, ?it/s]

Investing in stocks ['PACB', 'TTOO', 'STT', 'ISSC', 'HUM'] on 2021-01-29 with sentiments [0.57, 0.54, 0.49, 0.48, 0.48] and returns [0.09, 0.12, 0.01, 0.06, -0.02] on 2021-02-02
Average return for the day 2021-02-02 is 0.052
Balance on 2021-02-01 is 1142.7007014838798

Looking for articles on 2021-02-01 for Investment


 32%|███▏      | 2093/6602 [11:54<21:57,  3.42it/s]   

In [ ]:
try:
    import cPickle as pickle
except ImportError:  # Python 3.x
    import pickle

with open(f'news_from_{dates[0]}_to_{dates[-1]}.p', 'wb') as fp:
    pickle.dump(all_news, fp, protocol=pickle.HIGHEST_PROTOCOL)